# Logistic Regression Baseline model to Cancer Networks

In [1]:
data_dir = '/home/essharom/code/cancer-gnn-nf/data'

In [7]:
# Import packages
import os
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from pandas.api.types import CategoricalDtype
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix, fbeta_score

from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import fbeta_score
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
import numpy as np




In [9]:
df.head()

,gene_name,enhancers,gc19_pc.5utr,gc19_pc.3utr,gc19_pc.cds,gc19_pc.promCore,label
0,2,0.676548,0.864645,-0.000000,0.546244,1.134792,0
1,14,3.628010,2.448351,-0.000000,1.531436,2.163510,0
2,19,5.307124,3.010156,1.315560,0.533146,3.407497,0
3,23,1.553058,-0.000000,0.917732,6.733592,-0.000000,0
4,25,2.524617,3.279794,1.961659,2.545931,5.191230,1


In [34]:
data_pairs = [
    {
        'name': 'string',
        'networkFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_stringhc.tsv',
        'geneFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_fpancanall_nstringhc_lbailey.csv'
    },
    {
        'name': 'string_cosmic',
        'networkFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_stringhc.tsv',
        'geneFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_fpancanall_nstringhc_lcosmic.csv'
    },
    {
        'name': 'biogrid',
        'networkFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_biogridhc.tsv',
        'geneFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_fpancanall_nbiogridhc_lbailey.csv'
    },
    {
        'name': 'biogrid_cosmic',
        'networkFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_biogridhc.tsv',
        'geneFile': '/home/essharom/code/cancer-gnn-nf/data/entrez_fpancanall_nbiogridhc_lcosmic.csv'
    }
]

In [40]:
for dataset in data_pairs:
    print(dataset['name'])
    # Load your dataset
    df = pd.read_csv(dataset['geneFile']) 

string
   gene_name  enhancers  gc19_pc.5utr  gc19_pc.3utr  gc19_pc.cds  \
0          1  -0.000000     -0.000000      2.821174     0.276227   
1          2   0.676548      0.864645     -0.000000     0.546244   
2          9  -0.000000      1.605924     -0.000000     0.499488   
3         10  -0.000000     -0.000000     -0.000000    -0.000000   
4         12   2.772589      1.131268      2.845917     1.651073   

   gc19_pc.promCore  label  
0          0.396902      0  
1          1.134792      0  
2          1.217612      0  
3         -0.000000      0  
4         -0.000000      0  
string_cosmic
   gene_name  enhancers  gc19_pc.5utr  gc19_pc.3utr  gc19_pc.cds  \
0          1  -0.000000     -0.000000      2.821174     0.276227   
1          2   0.676548      0.864645     -0.000000     0.546244   
2          9  -0.000000      1.605924     -0.000000     0.499488   
3         10  -0.000000     -0.000000     -0.000000    -0.000000   
4         12   2.772589      1.131268      2.845917     

In [47]:
for dataset in data_pairs:
    print(dataset['name'])
    # Load your dataset
    df = pd.read_csv(dataset['geneFile']) 

    # Selecting features and target
    X = df[['enhancers', 'gc19_pc.5utr', 'gc19_pc.3utr', 'gc19_pc.cds','gc19_pc.promCore']]
    y = df['label']

    # Number of runs
    n_runs = 10
    bacc_scores = []

    for _ in range(n_runs):

        # Assuming X and y are your features and target variable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Initialize the scaler
        scaler = StandardScaler()

        # Fit the scaler on the training data
        scaler.fit(X_train)

        # Transform both the training and test data with the fitted scaler
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)


        # Creating and training the model
        model = LogisticRegression(C=0.01, penalty='l2', solver='newton-cg', class_weight='balanced', max_iter=1000)

        model.fit(X_train_scaled, y_train)

        # Making predictions
        predictions = model.predict(X_test_scaled)

        # Calculating Balanced Accuracy
        bacc = balanced_accuracy_score(y_test, predictions)
        bacc_scores.append(bacc)

    # Calculating average and standard deviation of BACC
    average_bacc = np.mean(bacc_scores)
    std_bacc = np.std(bacc_scores)

    print(f"Average Balanced Accuracy over {n_runs} runs: {average_bacc}")
    print(f"Standard Deviation of Balanced Accuracy: {std_bacc}")

string
Average Balanced Accuracy over 10 runs: 0.7186834036790903
Standard Deviation of Balanced Accuracy: 0.029271186281129836
string_cosmic
Average Balanced Accuracy over 10 runs: 0.6358778475038271
Standard Deviation of Balanced Accuracy: 0.02813505611033466
biogrid
Average Balanced Accuracy over 10 runs: 0.6971084565085023
Standard Deviation of Balanced Accuracy: 0.034505013353398346
biogrid_cosmic
Average Balanced Accuracy over 10 runs: 0.5987351991044413
Standard Deviation of Balanced Accuracy: 0.02535511704076008


In [12]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


lr = LogisticRegression(max_iter = 1000, class_weight="balanced")

solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['none', 'l1','elasticnet', 'l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)


grid_search = GridSearchCV(estimator=lr,
                           param_grid=grid,
                           n_jobs=-1,
                           cv=cv,
                           scoring= 'balanced_accuracy',
                           error_score=0)

grid_result = grid_search.fit(X_s, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_,
                             grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']

stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

Best: 0.710032 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'none', 'solver': 'newton-cg'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'none', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'none', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'lbfgs'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'elasticnet', 'solver': 'liblinear'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.709024 (0.064777) with: {'C': 100, 'penalty': 'l2', 'solver': 'l

/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/essharom/miniconda3/envs/jupyter_env/lib/python3.10/site-p

Best: 0.710032 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
